***
### <span style='color:green'> ME Algorithm  &emsp;&emsp; Feb, 2024 </span>
### <span style='color:Blue'> Phase 5 </span>
### <p> Yan-Bin Chen (陳彥賓) &emsp; yanbin@stat.sinica.edu.tw </p>
### <p> Institute of Statistical Science, Academia Sinica, Taipei, Taiwan.</p>  
---

In [1]:
import pandas
import numpy as np
import pickle
import os
import csv
import time
import datetime
import collections
from itertools import chain
from scipy.spatial.distance import squareform, pdist

# Input

In [2]:
# input
PATH4='../../phase3/data/ResNet18_PlantDisease_45K_Spec200_sampling.csv'
PATH5='../../phase3/data/embedded_data.pickle'
PATH6='../data/mergedseedclasslabels.txt'
PATH7='../../phase3/data/region_for_phase5.pickle'


# === parameters ===================================================
MNIST     = False
NUM_CASE  = 1
INTE_bool = False  #True: Integrate two networks VGG+ResNet    False: single network
SAVE_bool = True
ITE_FROM  = 5 # This setting is ONLY for Integration
REG_COLUMN = "Spec200"
RAW_2D_DATA = False
interpret_path='./case1/accu_history.csv'  #interprete the accuracy results
AMOUNT_ITE=3


if RAW_2D_DATA: # 2D
    from CNN_Modules import ME_CNN
else: # 1D
    from CNN_Modules_1D import ME_CNN
    

if (INTE_bool):
    ITE_START=ITE_FROM
    ITE_END=ITE_FROM+4
else:
    ITE_START=0
    ITE_END=5

# Define functions

In [3]:
def append_csv(x, path):
    with open(path,'a+', newline='') as f:
        csv_file = csv.writer(f)#   = f.write()
        csv_file.writerow(x)

In [4]:
# Only for single network. No necessary in Integrated networks
if (not INTE_bool):
    def create_image_0(PATH6, case_i):
        # ===================
        #
        #  prepare  merged_region_image_0
        #
        #====================
        # (A)
        #get "(1)merged_region"      only seed regions, no neighboring regions
        df = pandas.read_csv(PATH6, delim_whitespace=' ', header=0,  index_col=None)
        table = df.to_numpy()
        print("mergedseedclasslabels table")
        display(table)

        merged_region=[]
        for i in range(min(table.T[case_i+1]), max(table.T[case_i+1])+1):  #18 ---merge to --> 10
            addr=np.where(table.T[case_i+1]==i)[0] # 2nd column equal to 0(min),1,2,3...10(max); DO NOT consider 3rd column, which is hidden
            if(len(addr) and i>0): #if not empty and i=0 is the invalid seed region.
                merged_region.append(table[addr][:,0].tolist())
        print("merged_region")
        display(merged_region)


        # (B)
        #get "merged_reg_and_nei"
        #get "merged_reg_and_nei_image"
        #generate "merged_region_image_0.pickle"

        # (B_a)=== without neighbors ====
        #if ((DATASET == 2) or (DATASET == 4)): 
        ##20240105
        if (not True): 
            # ==== collect regions. No neighbors, just use merged regions ====
            merged_reg_and_nei=merged_region.copy()

            # ==== collect images ====
            img_temp=[]
            for i in range(len(merged_region)):
                addr=[]
                for j in range(len(merged_region[i])):
                    temp=np.where(all_region_index==merged_region[i][j])[0].tolist()   #tolist(): convert temp into list
                    addr=addr+temp
                    print(len(temp),end=' ')
                img_temp.append(addr)
                print("=",len(img_temp[i]))
            merged_reg_and_nei_image = img_temp.copy()


        # (B_b)=== with neighbors ==== 
        else: 
            with open(PATH7, 'rb') as f:
                pre_region, pre_reg_nei, pre_region_image_pure, pre_region_image= pickle.load(f)
            #    1reg         2reg+nei        1's img            2's img

            # ==== collect regions with neighbors====
            # remove duplicate  -->  https://stackoverflow.com/questions/9835762/how-do-i-find-the-duplicates-in-a-list-and-create-another-list-with-them
            merged_reg_and_nei=[]
            NUM_region=len(merged_region)
            for i in range(NUM_region):
                temp=[]
                for j in range(len(merged_region[i])):
                    idx=np.where(pre_region==merged_region[i][j])[0][0] 
                    temp=temp+pre_reg_nei[idx]
                    print(idx,pre_region[idx])
                merged_reg_and_nei.append(temp)


                #check whether it has duplicates
                if (len(merged_reg_and_nei[i]) != len(set(merged_reg_and_nei[i]))):
                    a=merged_reg_and_nei[i].copy()

                    # find the duplicate.
                    seen = set()
                    dupli                 = [x for x in a if (x in seen or seen.add(x))]
                    print("***duplicates:",dupli)

                    # keep fisrt one, remove succeeding duplicates.
                    seen = set()
                    merged_reg_and_nei[i] = [x for x in a if not (x in seen or seen.add(x))]  # a is the data to process; x is a working varialbe
                    print("unique:",merged_reg_and_nei[i])

                print("total",len(merged_reg_and_nei[i]),end="\n\n")


            print("\nmerged_reg_and_nei")
            for i in range(len(merged_reg_and_nei)):
                print(merged_reg_and_nei[i])


            # Collect images
            merged_reg_and_nei_image=[]
            for i in range(NUM_region):
                #search and add
                img=[]
                for j in range(len(merged_region[i])):
                    idx=np.where(pre_region==merged_region[i][j])[0][0]
                    print(len(pre_region_image[idx]),"(",idx,")",end=' ')
                    img=img+pre_region_image[idx] 
                print("=",len(img),end=" ")

                #check whether it has duplicates
                if (len(img) != len(set(img))):
                    img=list(set(img)) #remove duplicates
                    print("     **duplicate, shrink to",len(img),end="\n")  
                else:
                    print(end="\n")

                #append
                merged_reg_and_nei_image.append(img)

            print("\nmerged_reg_and_nei_image")
            for i in range(len(merged_reg_and_nei_image)):
                print(len(merged_reg_and_nei_image[i]),merged_reg_and_nei_image[i][:5],"...")

        # save
        if (SAVE_bool):
            with open(newpath+'/merged_region_image_0.pickle', 'wb') as f:
                pickle.dump([merged_reg_and_nei, merged_reg_and_nei_image], f)     

In [5]:
def CNN_part(PATH5,ITE):
    TRIALS          = 5

    savelog_path = newpath+'/' + 'log.txt'

    # ==== test_array ====
    with open(PATH5, 'rb') as f:
        test_array, test_label_answer = pickle.load(f)
        
    if RAW_2D_DATA: # 2D
        print("")
    else: # 1D
        test_array = np.expand_dims(test_array, axis = -1)

    
    #if((DATASET==2) or (DATASET==4)):
    #    test_array = np.expand_dims(test_array, axis = -1)
    #elif(DATASET==1):
    #    test_array = np.expand_dims(test_array, axis = -1)
    #    test_array /= 255
    #elif(DATASET==0):
    #    test_array /= 255
    #display(np.shape(test_array))


    with open(newpath+'/merged_region_image_'+str(ITE)+'.pickle', 'rb') as f:
        merged_reg_and_nei, merged_region_image = pickle.load(f)
    region_image=merged_region_image.copy()
    del merged_reg_and_nei


    NUM_region=len(region_image)
    print("NUM_region",NUM_region)


    from itertools import chain
    region_image_flatten=list(chain.from_iterable(region_image))
    print("number of clean images",len(region_image_flatten))


    ROUND_start = time.time()
    #========  merge ==========
    #prepare selected_region, region
    for n in range(1): #extra_original
    #   #reset
        region=region_image.copy()
        region=list(region)
        selected_region = list(range(NUM_region))  #[0,1,2, ... ,29]

        #merge
        if (n > 4):
            p1=comb[n-1][0]
            p2=comb[n-1][1]
            region[p1]=region[p1]+region[p2]
            region.pop(p2)
            selected_region.pop(-1)  # remove last region index
        #original
        else:  #n=0
            p1=0
            p2=0

        print("n, p1, p2", n, p1, p2)


        # ===== one CNN =============
        NUM_CLASSES = len(selected_region)  #NUM_CLASSES should be here to update for each loop

        # input image and label
        Input_img     = []
        Input_img_len = []
        for c,sel in enumerate(selected_region, start=0):
            Input_img = Input_img + list(region[sel])
            Input_img_len.append(len(region[sel])) #can only concatenate list (not "int") to list    
            
        # 20240319
        if RAW_2D_DATA: # 2D
            W           = np.shape(test_array[0])[0]
            H           = np.shape(test_array[0])[1]
            train_array = np.zeros((len(Input_img), W, H), dtype=float)
            for i in range (len(Input_img)):
                train_array[i] = test_array[Input_img[i]].reshape(W,H)
        else: # 1D
            W           = np.shape(test_array[0])[0]
            train_array = np.zeros((len(Input_img), W), dtype=float)
            for i in range (len(Input_img)):
                train_array[i] = test_array[Input_img[i]].reshape(W)
                  
        train_array = np.expand_dims(train_array, axis = -1)


        # fill up the training label to each training image
        current_train_label = np.zeros(len(train_array), dtype=int)  # Assign 0 to the label
        accum_base=0  #accumulate
        for label in range(1,NUM_CLASSES):
            sector = Input_img_len[label-1]
            accum_base = accum_base + sector  # sector is the sector length
            current_train_label[accum_base:] = label  # fill the label


        # CNN
        #===============================================
        one_predicted_results  = np.zeros((TRIALS, len(test_label_answer)), dtype=int)
        one_predict_percentage = np.zeros((TRIALS, len(test_label_answer), NUM_CLASSES), dtype=float)    
        model_history = np.zeros(TRIALS, dtype=list)
        print("NUM_CLASSES",NUM_CLASSES)
        print("current_train_label: ",list(set(current_train_label)))
        for r in range(TRIALS):  #10
            one_predicted_results[r], one_predict_percentage[r], model_history[r] = ME_CNN(
                    x_train     = train_array,
                    train_label = current_train_label,
                    test_array  = test_array,
                    true_answer = test_label_answer,
                    Num_Classes = NUM_CLASSES
                    )
            print(type(model_history))


            # ===== delete CNN tensors =====
            from keras import backend as K
            K.clear_session()
            import gc
            gc.collect()

            print("One CNN, r: ",r)
            ROUND_duration = time.time() - ROUND_start
            print("Computing Time: ", str(datetime.timedelta(seconds=ROUND_duration)))


        # === save to file ===
        #This is useless in phase IV. Prepare for further checking in the future.
        savefile_path = str(newpath) +  '/(classes=' + str(NUM_CLASSES)+')_n0_R' + str(p1) + '+R'+ str(p2) +'_trial' + str(n)+'_'+str(ITE)+'.pickle'  #extra_original
        with open(savefile_path, 'wb') as f:
            pickle.dump([Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history], f)

        savefile_path2 = str(newpath) +  '/(classes=' + str(NUM_CLASSES)+')_5_tests_simple_ITE'+str(ITE)+'.pickle'  #extra_original
        with open(savefile_path2, 'wb') as f:
            pickle.dump([one_predicted_results, one_predict_percentage], f)

        # === save to log ===    
        savelog = open(savelog_path, 'a+')
        print("\n", savefile_path, file = savelog)
        print("Saved parameters: Input_img, Input_img_len, one_predicted_results, one_predict_percentage", file = savelog) #0722

        # total time
        ROUND_duration = time.time() - ROUND_start
        print("Completion time: ", datetime.datetime.now(), file = savelog)
        print("Total Computing Time: ", str(datetime.timedelta(seconds=ROUND_duration)), file = savelog)

        savelog.close()

In [6]:
def statistic_method(PATH5,NUM_region,region_label,table_1D):
    with open(PATH5, 'rb') as f:
        test_array, test_label_answer = pickle.load(f)
    del test_array
    
    dist_table_truth=np.zeros((NUM_region,NUM_region),dtype=int)
    region_correct=[]
    region_amount=[]
    overall_correct=0
    overall_amount=0

    for i in range(NUM_region):
        #(1) input
        region_image=np.where(table_1D==i)[0]
        #region_image=merged_region_image[i].copy()
        
        #(2) establish confusion matrix
        for j in range(NUM_region):
            dist_table_truth[i][j]=len(np.where(test_label_answer[region_image]==j)[0]) #the number of images which equals to true answer 
        
        #(3) statisitc
        region_correct.append(dist_table_truth[i][region_label[i]])
        region_amount.append(len(region_image))
      
    #(4) statistic for overall
    overall_correct=sum(region_correct)
    overall_amount=sum(region_amount)

    return region_correct,region_amount,overall_correct,overall_amount,dist_table_truth

In [7]:
def statistic(PATH5,ITE):
    # input 1:
    # (1)merged_region_image_(ITE)
    with open(newpath+'/merged_region_image_'+str(ITE)+'.pickle', 'rb') as f:
        merged_reg_and_nei, merged_region_image = pickle.load(f)
    del merged_reg_and_nei
    NUM_region=len(merged_region_image)
    
    # (2)test_label_answer
    with open(PATH5, 'rb') as f:
        test_array, test_label_answer = pickle.load(f)
    del test_array

    # (3)get consistent result table
    with open(newpath+'/(classes=' + str(NUM_region) + ')_5_tests_simple_ITE'+str(ITE)+'.pickle', 'rb') as f:
        one_predicted_results, one_predict_percentage = pickle.load(f)
    del one_predict_percentage
    LENGTH=np.shape(one_predicted_results)[1]
    Original_result=np.zeros(LENGTH,dtype=int)
    for i in range(LENGTH):
        if (len(set(one_predicted_results.T[i])) == 1):  # (***)
            Original_result[i]=one_predicted_results[0][i]
        else:
            Original_result[i]=-1
 
    # (4) get region_label 
    region_label=[] #true label by selecting dominate ones
    for i in range(NUM_region):
        region_image=merged_region_image[i].copy()
        region_label.append(collections.Counter(test_label_answer[region_image]).most_common()[0][0])  #images --> true label --> most_common label


   #========================================     
   # (1)train + test
    a2,b2,c2,d2,e2=statistic_method(PATH5,NUM_region,region_label,Original_result)
    na2=np.asarray(a2)
    nb2=np.asarray(b2)
    nc2=np.asarray(c2)
    nd2=np.asarray(d2)
    all_num=len(Original_result)
    append_csv([ITE, c2, d2,      round(nc2/nd2    ,3), "5con over all, but 5-consensus"], csv_path1)
    append_csv([ITE, c2, all_num, round(nc2/all_num,3), "5con over all"], csv_path1)
 

        
    # (2)train
    train_results=-1*np.ones(LENGTH,dtype=int)
    for i in range(NUM_region):
        images=merged_region_image[i]
        train_results[images]=i
        print("num of merged_region_image",i,len(merged_region_image[i]))
    print(collections.Counter(train_results))
        
    a1,b1,c1,d1,e1=statistic_method(PATH5,NUM_region,region_label,train_results)
    na1=np.asarray(a1)  #region_correct
    nb1=np.asarray(b1)  #region_amount
    nc1=np.asarray(c1)  #overall_correct
    nd1=np.asarray(d1)  #overall_amount
    all_num=len(Original_result)
    append_csv([ITE, c1, d1, round(nc1/nd1,3), "5con over trained"], csv_path1)
    append_csv([ITE, c1, all_num, round(nc1/all_num,3), "5con over all"], csv_path1)
    
        
    # (3)test
    # remove training data(good images), only check test data(bad images)
    from itertools import chain
    used_image=merged_region_image.copy()
    used_image=list(chain.from_iterable(used_image))
    Original_result2=Original_result.copy()
    Original_result2[used_image]=-2

        
    a4,b4,c4,d4,e4=statistic_method(PATH5,NUM_region,region_label, Original_result2)
    na4=np.asarray(a4)
    nb4=np.asarray(b4)
    nc4=np.asarray(c4)
    nd4=np.asarray(d4) #this is len(Original_result)-len(used_image)-len(unconsistent)
    untrain=len(Original_result)-len(used_image)
    all_num=len(Original_result)
    append_csv([ITE, c4, untrain, round(nc4/untrain, 3), "5con over untrained, but 5-consensus"], csv_path1)
    append_csv([ITE, c4, all_num, round(nc4/all_num, 3), "5con over untrained (unclean)"], csv_path1)

        
    # (4)set majority as label for each image        
    # set majority from 5 trias as label for each image
    predicted_results_major=np.zeros(LENGTH,dtype=int)
    for i in range(LENGTH):
        predicted_results_major[i]=collections.Counter(one_predicted_results.T[i]).most_common()[0][0]
    

    a3,b3,c3,d3,e3=statistic_method(PATH5,NUM_region,region_label,predicted_results_major)
    na3=np.asarray(a3)
    nb3=np.asarray(b3)
    nc3=np.asarray(c3)
    nd3=np.asarray(d3) #this is all in majority criterion
    append_csv([ITE, c3, d3, round(nc3/nd3    ,3), "majo over all"], csv_path1)

In [8]:
def merged_and_expand(PATH5,ITE):
# all4.     
    # load
    with open('./region_initials.pickle', 'rb') as f:
        all_region_index, all_region_image = pickle.load(f)
    MAX_region = max(all_region_index)

    with open(newpath+'/merged_region_image_'+str(ITE)+'.pickle', 'rb') as f:
        merged_reg_and_nei, merged_region_image = pickle.load(f)
    NUM_region = len(merged_reg_and_nei) # NUM_region is the number of clusters

    with open(newpath+'/(classes='+str(NUM_region)+')_5_tests_simple_ITE'+str(ITE)+'.pickle', 'rb') as f:
        one_predicted_results, one_predict_percentage = pickle.load(f)
    del one_predict_percentage

    with open(PATH5, 'rb') as f:
        test_array, test_label_answer = pickle.load(f)
    del test_array


    # choose absolutely consistent images
    NUM_test=np.shape(one_predicted_results)[1]
    Original_result=np.zeros(NUM_test,dtype=int)

    # (***)
    # As set contains only unique elements, so convert the list to set.
    # If set size is 1 then it means all elements in given list are same
    for i in range(NUM_test):
        if (len(set(one_predicted_results.T[i])) == 1):  # (***)
            Original_result[i]=one_predicted_results[0][i]
        else:
            Original_result[i]=-1
    
    used_img=list(chain.from_iterable(merged_region_image))
    used_img=np.sort(used_img)
    working_img = np.asarray(list(  set(range(NUM_test))-set(used_img)  ))  #working_img means the unclean ones for working on the further adding process
    print("===========  ITE =",ITE, "  ===========")    
    print("used_img",len(used_img), len(set(used_img)))    
    print("working_img(=other images=unclean images)",len(working_img), len(set(working_img)))

    # save clean and unclean images
    if (SAVE_bool):
        with open(newpath + '/clean_and_unclean_image_ITE='+str(ITE)+'.pickle', 'wb') as f:
            pickle.dump([used_img, working_img], f) #used_img is clean, working_img is others

    # other_regions
    # ==== Process of other regions. Generate "other_regions" ====
    merged_reg_and_nei_flatten=list(chain.from_iterable(merged_reg_and_nei))
    print("merged regions", len(merged_reg_and_nei_flatten), len(set(merged_reg_and_nei_flatten)))
    other_regions       = list(  set(range(1,MAX_region+1))-set(merged_reg_and_nei_flatten)  ) #region index exclude used regions. 1 to 200.
    print("other_regions",len(other_regions), len(set(other_regions)))
    
    dmn_img             = [] # Index of dmn_img is consistent with other_regions
    NUM_other_regions   = len(other_regions) # number of clusters in other regions
    dist_table_truth    = np.zeros((NUM_other_regions,NUM_region),dtype=int)
    p_reg_label_dmn     = np.zeros(NUM_other_regions,dtype=int)   #one value. dominate label in predicted lagels.
    grd_reg_answer_dmn  = np.zeros(NUM_other_regions,dtype=int)   #one value. dominate label in true answers.
    p_reg_dmn_rate      = np.zeros(NUM_other_regions,dtype=float) #one value. dominate ratio in a region

    #(1) other_regions      --> establish all other region table 
    for i,region_name in enumerate(other_regions): #check all other regions

        #(a)===== predicted images (multiple values) =====
        p_img        = all_region_image[region_name-1] # In this region, get their images. "region_name-1" is due to region index starts from 1 to 200
        p_img_label  = Original_result[p_img]   # Predicted labels in the region.
        p_img_total  = len(p_img)
        # the value of predicted labels is the index of trainning region. These indices are the labels
        # but these p_img_answer are predicted, may not always be the truth.
        if not p_img: # if p_img is empty, skip this loop
            dmn_img.append([])
            continue

        #(b)===== region dominate; one value =====
        #region label
        p_reg_label_dmn[i] = collections.Counter(p_img_label).most_common()[0][0] # one value
        # region dominate rate
        if(p_reg_label_dmn[i]>=0):
            p_reg_dmn_rate[i] = collections.Counter(p_img_label).most_common()[0][1]/p_img_total
        else:              # means invalid label
            p_reg_dmn_rate[i] = 0


        #(c)==== ground truth =====
        grd_label                 = test_label_answer[p_img]  #multiple values
        grd_reg_answer_dmn[i]     = collections.Counter(grd_label).most_common()[0][0] #one value


        #(d)==== establish confusion table=====
        for j in range(NUM_region):
            dist_table_truth[i][j]=len(np.where(grd_label==j)[0])


        #(e)=== collect dominated images =============
        addr2=np.where( (p_img_label==p_reg_label_dmn[i]) & (p_img_label>=0) )[0] # ignore -1 which are non-consistency
        #         the labels which  == 7               the labels which >= 0
        temp=[]
        for k in range(len(addr2)):
            temp.append(p_img[addr2[k]])
        dmn_img.append(temp)
        #=============================================


    df1 = pandas.DataFrame({"other index":other_regions}) # 1 to 200   other region index
    df2 = pandas.DataFrame({"pred label":p_reg_label_dmn})      
    df4 = pandas.DataFrame({"truth":grd_reg_answer_dmn})
    df6 = pandas.DataFrame({"rate":np.round(p_reg_dmn_rate,2)})
    df7 = pandas.DataFrame(dist_table_truth)
    entire_table=pandas.concat([df1, df2, df4, df6, df7], axis=1)
    print("All other regions")
    display(entire_table)



    #(2)get regions according to conditions
    NN=5 #choose top 5 regions
    RATE=0.7
    candidate_reg_by_top_NN=[]

    # === get candidate regions by the order of dmn label 0 to 9 ====
    for i in range(NUM_region):
        # (2-1) ==== select region by rate > 0.7 and top 5 ====
        index     = np.where(p_reg_label_dmn==i)[0] # index is the index of other_regions(0~183), rather than original entire region index 1 to 200        
        working_table = entire_table.iloc[index]
        working_table = working_table.sort_values(by=['rate'], ascending=False)
        working_table = working_table.loc[working_table['rate'] > RATE]  #rate > 0.7
        NUM_region_in_one_class = len(working_table.iloc[:NN])  #top 5
               
        # (2-2) ==== get candidate regions ====
        # get top N records; save only the column 'other_reg', and transfer it to list from DataFrame by "tolist()"
        candidate_reg_by_top_NN.append(working_table[:NN]['other index'].tolist())
         
    #(3) add regions and images
    for i in range(NUM_region):
        added_img=[]
        if (len(candidate_reg_by_top_NN[i])>0):
            for j in range(len(candidate_reg_by_top_NN[i])):
                reg_addr  = np.where( np.array(other_regions)==candidate_reg_by_top_NN[i][j] )[0][0].tolist()
                added_img = added_img + dmn_img[reg_addr]
            # (3-1) add image
            temp=len(merged_region_image[i])
            merged_region_image[i] = merged_region_image[i] + added_img
            merged_region_image[i] = list(set(merged_region_image[i]))
            img_amount=len(merged_region_image[i])-temp
            
            # (3-2) add region
            merged_reg_and_nei[i]  = merged_reg_and_nei[i] + candidate_reg_by_top_NN[i]
            
            # (3-3) print out
            print("added label, regions, img amount:", set(Original_result[added_img]), candidate_reg_by_top_NN[i], img_amount)

            
    # (4) collect residual images
    # This works only for CIFAR10. All images in the MNIST and MNIST-TRAN are clean. No this issue.        
    #20240105
    if (not MNIST):
    #if ((DATASET==2) or (DATASET==4)):
        if (len(list(chain.from_iterable(candidate_reg_by_top_NN))) == 0):  #if no extra regions
            #20240105
            if (True):
            #if(DATASET==4):
                df = pandas.read_csv(PATH4)
                tSNE_table = df.to_numpy()[:,:3]
            else:
                df = pandas.read_csv(PATH8)
                tSNE_table = df.to_numpy()
            print("tSNE_table",np.shape(tSNE_table))

            working_table=tSNE_table[working_img]
            pairwise_dist=squareform(pdist(working_table, 'euclidean'))
            print("pairwise_dist",np.shape(pairwise_dist)) #value of data point, rather than image index

            TopN=10
            M=len(working_img)
            nei_table_images  = np.zeros((M,TopN),dtype=int)  #contain top 10 images
            nei_table_label   = np.zeros((M,TopN),dtype=int)
            working_img_label = np.zeros(M,dtype=int)
            for i in range(M):   
                # fill up top 10 
                addr=np.argsort(pairwise_dist[i])
                for j in range(TopN):
                    nei_table_images[i][j]=working_img[ addr[j+1] ] #Ignore first one. First one is itself
                    nei_table_label[i][j] =Original_result[nei_table_images[i][j]]
                # consistent
                if (len(set(nei_table_label[i])) == 1): #only get the one which is entire consistent
                    working_img_label[i]=nei_table_label[i][0]
                else:
                    working_img_label[i]=-1

            print("nei_table_images",np.shape(nei_table_images))
            print("working_img_label",working_img_label)

            new_img=[] # just  for monitoring
            for i in range(NUM_region):
                addr=np.where(working_img_label==i)[0].tolist()
                new_img.append(working_img[addr])
                merged_region_image[i].extend(working_img[addr])
            print("add residuals ",len(list(chain.from_iterable(new_img))))
            print("number of next merged_region_image", len(list(chain.from_iterable(merged_region_image))))
        else:
            print("Not getting into residuals")

    #save
    if (SAVE_bool):
        with open(newpath + '/merged_region_image_'+str(ITE+1)+'.pickle', 'wb') as f:
            pickle.dump([merged_reg_and_nei, merged_region_image], f)

# Makeup region_initials.pickle
#### For both single network and integrate network

In [9]:
df = pandas.read_csv(PATH4)
display(df.head())
#all_region_index = df.to_numpy().T[REGION_INDEX_LOC].astype(int)
#print(len(all_region_index))
all_region_index  = df[REG_COLUMN].to_numpy().astype(int)
print(len(all_region_index))
print("all_region_index\n",all_region_index[:5])

all_region_image=[]
MAX_region=max(all_region_index)
for i in range(MAX_region):
    addr=list(np.where(all_region_index==i+1)[0])
    all_region_image.append(addr)    

#save
if (SAVE_bool):
    with open('./region_initials.pickle', 'wb') as f:
        pickle.dump([all_region_index, all_region_image], f)

,X1,X2,X3,Class,Label,Spec200
0,1.230895,-11.513212,18.781408,Cherry,2,177
1,3.190353,-14.243707,14.359673,Cherry,2,177
2,1.832576,-12.109021,15.785449,Cherry,2,177
3,1.880559,-12.113366,15.720321,Cherry,2,177
4,5.759654,-13.821998,15.943708,Cherry,2,177


13076
all_region_index
 [177 177 177 177 177]


# Main function

In [10]:
def interpret_accu_results(path, AMOUNT_ITE):
    df = pandas.read_csv(path)
    label_table = df.to_numpy()
    NUM_CRI=7  #number of our accuracy criteriors, now is 7
    
    criterion_string=\
    [ "correct in 5-consensus\n------------------------------------\n5-consensus\n",
     "correct in 5-consensus\n------------------------------------\nall\n",
     "correct in train in 5-consensus\n------------------------------------\ntrain in 5-consensus\n",
     "correct in train in 5-consensus \n------------------------------------\nall\n",
     "correct in test in 5-consensus\n------------------------------------\ntest in 5-consensus\n",
     "correct in test in 5-consensus\n------------------------------------\nall\n",
      "correct\n------------------\nall\n",
    ]
    for SHIFT in range (NUM_CRI):
        if (SHIFT+1==1):
            print("(overall 5-consensus)")
        elif (SHIFT+1==3):
            print("(clean)")
        elif (SHIFT+1==5):
            print("(unclean)")
        elif (SHIFT+1==7):
            print("(majority)")
        print("criterion", SHIFT+1)
        print(criterion_string[SHIFT])
        for i in range(AMOUNT_ITE):
            print("ITE",label_table[NUM_CRI*i+SHIFT].T[0], "   ",label_table[NUM_CRI*i+SHIFT].T[1],"/", label_table[NUM_CRI*i+SHIFT].T[2], "=",label_table[NUM_CRI*i+SHIFT].T[3])
        print("\n\n\n")

In [11]:
for case_i in range(NUM_CASE):

    #===== create folder case1, case2, case3...
    print("case=",case_i+1)
    newpath = './case' + str(case_i+1)
    if (not INTE_bool):
        if not os.path.exists(newpath):   #No necessary in Integration
            os.makedirs(newpath)
    
    #==== open csv 1
    csv_path1 = newpath+'/' + 'accu_history.csv'
    with open(csv_path1, 'a', newline='') as f:
        csv_file = csv.writer(f)
        csv_file.writerow(['ITE', 'correct', 'denominator', 'accu', 'description'])

# 1.
    if (not INTE_bool):
        create_image_0(PATH6, case_i)   #No necessary in Integration


    for ITE in range(ITE_START, ITE_END):
# 2. CNN
        CNN_part(PATH5,ITE)

# 3. statistic
        statistic(PATH5,ITE)

# 4. merged_and_expand 
        merged_and_expand(PATH5,ITE)

case= 1
mergedseedclasslabels table


array([[ 26,   6,   6],
       [182,   1,   1],
       [169,   7,   7],
       [124,   2,   2],
       [ 29,   4,   4],
       [  1,   3,   3],
       [ 57,   5,   5],
       [ 76,   7,   8],
       [ 53,   8,   9],
       [ 52,   2,   2],
       [183,   1,   1],
       [ 59,   8,   9],
       [ 24,   4,   4],
       [164,   9,   7]], dtype=int64)

merged_region


[[182, 183], [124, 52], [1], [29, 24], [57], [26], [169, 76], [53, 59], [164]]

1 182
10 183
total 2

3 124
9 52
total 2

5 1
total 6

4 29
12 24
total 2

6 57
total 1

0 26
total 6

2 169
7 76
total 2

8 53
11 59
total 2

13 164
total 1


merged_reg_and_nei
[182, 183]
[124, 52]
[1, 11, 12, 14, 15, 17]
[29, 24]
[57]
[26, 3, 4, 5, 6, 8]
[169, 76]
[53, 59]
[164]
286 ( 1 ) 177 ( 10 ) = 463 
282 ( 3 ) 524 ( 9 ) = 806 
216 ( 5 ) = 216 
169 ( 4 ) 183 ( 12 ) = 352 
143 ( 6 ) = 143 
96 ( 0 ) = 96 
166 ( 2 ) 177 ( 7 ) = 343 
472 ( 8 ) 501 ( 11 ) = 973 
209 ( 13 ) = 209 

merged_reg_and_nei_image
463 [7306, 7307, 7316, 7317, 7338] ...
806 [7309, 7373, 7390, 7430, 7433] ...
216 [1247, 1254, 1255, 1257, 1258] ...
352 [11820, 11829, 11831, 11837, 11838] ...
143 [11671, 11672, 11673, 11674, 11675] ...
96 [1246, 1253, 1256, 1264, 1269] ...
343 [4616, 4694, 4700, 4761, 4763] ...
973 [11793, 11821, 11822, 11827, 11828] ...
209 [1844, 1845, 1846, 1847, 1848] ...
NUM_region 9
number of clean images 3601
n, p1, p2 0 0 0
NUM_CLASSES 9
current_train_label:  [0, 1, 2, 3, 4, 5, 6, 7, 8]


...

Epoch 28/80
9240/9240 [==============================] - 1s 87us/step - loss: 5.5370e-06 - accuracy: 1.0000 - val_loss: 3.3633e-04 - val_accuracy: 1.0000
Epoch 29/80
9240/9240 [==============================] - 1s 85us/step - loss: 5.1317e-06 - accuracy: 1.0000 - val_loss: 3.0687e-04 - val_accuracy: 1.0000
Epoch 30/80
9240/9240 [==============================] - 1s 86us/step - loss: 4.7245e-06 - accuracy: 1.0000 - val_loss: 3.1182e-04 - val_accuracy: 1.0000
Epoch 31/80
9240/9240 [==============================] - 1s 88us/step - loss: 4.4150e-06 - accuracy: 1.0000 - val_loss: 2.9487e-04 - val_accuracy: 1.0000
Epoch 32/80
9240/9240 [==============================] - 1s 87us/step - loss: 4.0672e-06 - accuracy: 1.0000 - val_loss: 2.9813e-04 - val_accuracy: 1.0000
Epoch 33/80
9240/9240 [==============================] - 1s 86us/step - loss: 3.8114e-06 - accuracy: 1.0000 - val_loss: 2.8861e-04 - val_accuracy: 1.0000
Epoch 34/80
9240/9240 [==============================] - 1s 88us/step - loss

[6 7 6 ... 7 3 7]
<class 'numpy.ndarray'>
One CNN, r:  3
Computing Time:  0:04:29.374261
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1000, 6)           36        
_________________________________________________________________
average_pooling1d_1 (Average (None, 500, 6)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 496, 16)           496       
_________________________________________________________________
average_pooling1d_2 (Average (None, 248, 16)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 244, 120)          9720      
_________________________________________________________________
flatten_1 (Flatten)          (None, 29280)             0         
_______________________________

9240/9240 [==============================] - 1s 85us/step - loss: 1.8485e-06 - accuracy: 1.0000 - val_loss: 2.9878e-04 - val_accuracy: 1.0000
Epoch 45/80
9240/9240 [==============================] - 1s 86us/step - loss: 1.7591e-06 - accuracy: 1.0000 - val_loss: 2.9997e-04 - val_accuracy: 1.0000
Epoch 46/80
9240/9240 [==============================] - 1s 87us/step - loss: 1.6489e-06 - accuracy: 1.0000 - val_loss: 3.0032e-04 - val_accuracy: 1.0000
Epoch 47/80
9240/9240 [==============================] - 1s 85us/step - loss: 1.5557e-06 - accuracy: 1.0000 - val_loss: 2.8740e-04 - val_accuracy: 1.0000
Epoch 48/80
9240/9240 [==============================] - 1s 88us/step - loss: 1.4722e-06 - accuracy: 1.0000 - val_loss: 3.0150e-04 - val_accuracy: 1.0000
Epoch 49/80
9240/9240 [==============================] - 1s 86us/step - loss: 1.3870e-06 - accuracy: 1.0000 - val_loss: 2.7732e-04 - val_accuracy: 1.0000
Epoch 50/80
9240/9240 [==============================] - 1s 86us/step - loss: 1.3285e-06

,other index,pred label,truth,rate,0,1,2,3,4,5,6,7,8
0,2,0,0,0.0,0,0,0,0,0,0,0,0,0
1,7,-1,8,0.0,0,0,0,0,0,0,0,0,65
2,9,0,0,0.0,0,0,0,0,0,0,0,0,0
3,10,0,0,0.0,0,0,0,0,0,0,0,0,0
4,18,0,0,0.0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,192,0,0,0.0,0,0,0,0,0,0,0,0,0
133,193,0,0,0.0,0,0,0,0,0,0,0,0,0
134,195,0,0,0.0,0,0,0,0,0,0,0,0,0
135,196,0,0,0.0,0,0,0,0,0,0,0,0,0


tSNE_table (13076, 3)
pairwise_dist (2809, 2809)
nei_table_images (2809, 10)
working_img_label [-1 -1 -1 ... -1  0 -1]
add residuals  306
number of next merged_region_image 10573


# Interpret the accuracy results

In [12]:
interpret_accu_results(interpret_path, AMOUNT_ITE)

(overall 5-consensus)
criterion 1
correct in 5-consensus
------------------------------------
5-consensus

ITE 0     8149 / 11565 = 0.705
ITE 1     9985 / 11595 = 0.861
ITE 2     10220 / 12000 = 0.852




criterion 2
correct in 5-consensus
------------------------------------
all

ITE 0     8149 / 13076 = 0.623
ITE 1     9985 / 13076 = 0.764
ITE 2     10220 / 13076 = 0.782




(clean)
criterion 3
correct in train in 5-consensus
------------------------------------
train in 5-consensus

ITE 0     3264 / 3601 = 0.9059999999999999
ITE 1     7217 / 7824 = 0.922
ITE 2     7934 / 8876 = 0.894




criterion 4
correct in train in 5-consensus 
------------------------------------
all

ITE 0     3264 / 13076 = 0.25
ITE 1     7217 / 13076 = 0.552
ITE 2     7934 / 13076 = 0.607




(unclean)
criterion 5
correct in test in 5-consensus
------------------------------------
test in 5-consensus

ITE 0     4886 / 9475 = 0.516
ITE 1     2772 / 5252 = 0.528
ITE 2     2287 / 4200 = 0.545




criterion 6
co

In [13]:
print("done")

done
